In this demo, we will experiment with different sets of parameters, and how this affects the number of slots available for SIMD operations, and the serialized size of encrypted inputs.

In [1]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

In [2]:
from pysheep import sheep_client

DB LOCATION IS /frontend/pysheep/sheep.db


In [3]:
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
sheep_client.new_job()

{'content': '', 'status_code': 200}

Lets use HElib_F2 and vary the BaseParamSet parameter (which in turn chooses pre-set values for other parameters):

In [10]:
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int8_t")
sheep_client.get_parameters()

{'content': {'BaseParamSet': 0,
  'BitsPerLevel': 25,
  'Bootstrap': 0,
  'HammingWeight': 128,
  'Levels': 7,
  'c': 2,
  'd': 12,
  'g1': 71,
  'g2': 76,
  'g3': 0,
  'm': 105,
  'm1': 3,
  'm2': 35,
  'm3': 0,
  'ord1': 2,
  'ord2': 2,
  'ord3': 0,
  'p': 65537,
  'phim': 48},
 'status_code': 200}

With BaseParamSet=0, how many slots for SIMD operations do we get?

In [11]:
num_slots = sheep_client.get_nslots()['content']['nslots']
num_slots

4

And how big is the ciphertext if we encrypt one input (which is a vector of 4 values since we have 4 slots)?

In [12]:
size=sheep_client.encrypt_and_serialize([1,2,3,4])["content"]["size"]


In [13]:
BaseParamSet=[]
nslots = []
ctsize = []
for i in range(5):
    sheep_client.set_parameter({"BaseParamSet": i})
    BaseParamSet.append(i)
    nslots.append(sheep_client.get_nslots()['content']['nslots'])
    ctsize.append(sheep_client.encrypt_and_serialize([1,2,3,4])['content']['size'])
    

'{"ciphertext":"[65537 0.3230066879e15 [0 1 2 3] 2\\n[[[0 1 2 3]\\n [16616949 10237461 10253213 3873330 6853815 1981362 2775556 2402920 10932781 7691349 4841223 12888948 10411499 15999334 581167 8850999 8615674 9555058 5595916 4476334 12215752 572088 13839104 15299673 267339 1843465 5085991 7324275 10193627 5880084 13344348 8375023 1853357 6124952 5446053 12165967 12778278 9878323 2361645 14683301 15391883 5423515 6872152 17435245 5327312 15342230 15717987 3069678]\\n [894641360682375 744323691242100 1097483707902284 354943320358083 153170526737680 287102831162560 730198562479636 795292165226757 112342053748189 912412206527613 711289931463040 63689832339375 505504682010647 662815655406532 887817247398622 347881106875118 224603841469058 22091677664810 193172679483366 39369597074995 1069467304819139 937255816487790 766019398066124 419690794437632 1072618055514494 348605149219534 614496219357360 814174162513457 789844621511107 152198874609112 541108414161565 836156779892683 42253337358217